In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
#import models
from tqdm import tqdm
import train_reg
import config
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F

/accounts/grad/xsli/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/accounts/grad/xsli/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
dset = 'clath_aux+gak_a7d2_new'
df = data.get_data(dset=dset)

## add regression response, downsample tracks

In [4]:
length = 40
df = train_reg.add_sig_mean(df) # regression response
df['X_same_length'] = [features.downsample(df.iloc[i]['X'], length) for i in range(len(df))] # downsampling

## extract feature names, get training data

In [5]:
feat_names = data.get_feature_names(df)
feat_names = [x for x in feat_names 
                      if not x.startswith('sc_') 
                      and not x.startswith('nmf_')
                      and not x in ['center_max', 'left_max', 'right_max', 'up_max', 'down_max',
                                   'X_max_around_Y_peak', 'X_max_after_Y_peak', 'X_quantiles',
                                   'X_d1', 'X_d2', 'X_d3', 'slope_end', 'X_same_length'
                                   ]
                      and not x.startswith('pc_')
                      and not 'log' in x
                      and not 'binary' in x
                     ]

In [6]:
df_train = df[df.cell_num.isin(config.DSETS[dset]['train'])] 
df_train = df_train[df_train.lifetime > 15]
df_train_nonna = df_train[feat_names + ['X_same_length', 'cell_num', 'Y_sig_mean']]
df_train_nonna = df_train_nonna.dropna() 

## train fully connected network

In [ ]:
train_reg.train_reg(df_train_nonna, 
                    feat_names=['X_same_length'] + feat_names,#['lifetime', 'X_max', 'mean_square_displacement'], 
                    model_type='fcnn', 
                    outcome_def='Y_sig_mean',
                    out_name=f'results/regression/Nov23_{dset}_Y_fcnn.pkl',
                    fcnn_hidden_neurons=20,
                    fcnn_epochs=2000)